In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
is_sDCA = 0
is_new = 1
do_scale_folder = 0

In [3]:
isigma = 2
N_centr = 6
N_options = 3
if is_new: N_options = 6
N_vtx_bins = 4
phi_v_low,phi_v_high,pt_low,pt_high = 1,-1,1,-1
centralities = ["0-88%","0-20%","20-40%","40-60%","60-93%","40-93%"]
centrality = [0,9,0,1,2,3,4,5,6,9,7,9]
colors=[1,2,4,6,root.kGreen+2]
file_path="input/"
file_name="m_ee_Run14AuAu_6th_new_19389_1050runs.root"
#m_ee_Run14AuAu_3rd_new_19383_1078runs
#m_ee_Run14AuAu_29th_test_19207_950runs m_ee_Run14AuAu_30th_test_19211_1078runs
#m_ee_Run14AuAu_27th_test_19200_1062runs m_ee_Run14AuAu_28th_test_19204_450runs
#m_ee_Run14AuAu_25th_test_19197_1000runs m_ee_Run14AuAu_26th_test_19198_1074runs
#m_ee_Run14AuAu_23rd_test_19184_500runs m_ee_Run14AuAu_24th_test_19192_980runs
#m_ee_Run14AuAu_21th_test_19127_970runs m_ee_Run14AuAu_22th_test_19180_1078runs
#m_ee_Run14AuAu_19th_test_19117_1016runs m_ee_Run14AuAu_20th_test_19121_999runs
#m_ee_Run14AuAu_17th_test_19114_790runs m_ee_Run14AuAu_18th_test_19115_1073runs
#m_ee_Run14AuAu_15th_test_19107_1080runs m_ee_Run14AuAu_16th_test_19109_920runs
#m_ee_Run14AuAu_twelfth_test_19102_920runs m_ee_Run14AuAu_bakers_dozen_test_19104_900runs
#m_ee_Run14AuAu_eleventh_test_19100_800runs m_ee_Run14AuAu_tenth_test_19097_800runs
#"m_ee_Run14AuAu_eighth_test_19090_900runs m_ee_Run14AuAu_ninth_test_19090_540runs.root"
#"m_ee_Run14AuAU_fourth_test_19072_800runs.root"#"m_ee_Run14AuAu_seventh_test_19089_720runs.root"

In [4]:
hists_ep_FG_read, hists_ep_BG_read, hists_ee_FG_read, hists_ee_BG_read = [], [], [], []
hist_stat_read=[]
infile = root.TFile.Open(file_path+file_name, "read")

for iOpt in range(0,N_options):
    histo_name = [f"inv_mass_ee_DCA_V{iOpt}_",f"inv_mass_ee_sDCA_V{iOpt}_"]
    histo_name_like = [f"delt_phi_ee_DCA_V{iOpt}_",f"delt_phi_ee_sDCA_V{iOpt}_"]
    print(histo_name)
    hists_ep_FG_read_centr, hists_ep_BG_read_centr, hists_ee_FG_read_centr, hists_ee_BG_read_centr = [], [], [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        hist_FG = infile.Get("c00_z00_r00/"+histo_name[is_sDCA]+"FG12")
        hist_FG.Add(hist_FG,-1)
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = infile.Get("c00_z00_r00/"+histo_name[is_sDCA]+"BG12")
        hist_BG.Add(hist_BG,-1)
        hist_BG.SetDirectory(root.nullptr)
        hist_FG_like = infile.Get("c00_z00_r00/"+histo_name_like[is_sDCA]+"FG11")
        hist_FG_like.Add(hist_FG_like,-1)
        hist_FG_like.SetDirectory(root.nullptr)
        hist_BG_like = infile.Get("c00_z00_r00/"+histo_name_like[is_sDCA]+"BG11")
        hist_BG_like.Add(hist_BG_like,-1)
        hist_BG_like.SetDirectory(root.nullptr)
        hist_stat = infile.Get("c00_z00_r00/PoolStatistics")
        hist_stat.Add(hist_stat,-1)
        hist_stat.SetDirectory(root.nullptr)
        for i in range(centrality[icent*2],centrality[icent*2+1]+1):
            for j in range(N_vtx_bins):
                for k in range(is_sDCA,1+is_sDCA):
                    dirname=f"c0{i}_z0{j}_r00/"
                    if i>9: dirname=f"c{i}_z0{j}_r00/"
                    hist_FG_local = infile.Get(dirname+histo_name[k]+"FG12")
                    hist_BG_local = infile.Get(dirname+histo_name[k]+"BG12")
                    hist_FG_like_local = infile.Get(dirname+histo_name_like[k]+"FG11")
                    hist_BG_like_local = infile.Get(dirname+histo_name_like[k]+"BG11")
                    hist_FG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"FG22"))
                    hist_BG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"BG22"))
                    eta_FG_porj = hist_FG_like_local.ProjectionX(f"FG_loc_{icent}_{i*20+j}",phi_v_low,phi_v_high,pt_low,pt_high)
                    eta_BG_porj = hist_BG_like_local.ProjectionX(f"BG_loc_{icent}_{i*20+j}",phi_v_low,phi_v_high,pt_low,pt_high)

                    left = eta_FG_porj.FindBin(1.4)
                    right = eta_FG_porj.FindBin(1.8)+1

                    scale = eta_FG_porj.Integral(left,right)/(eta_BG_porj.Integral(left,right)+0.1)
                    if not do_scale_folder: scale = 1
                    
                    hist_FG.Add(hist_FG_local)
                    hist_BG.Add(hist_BG_local,scale)
                    hist_FG_like.Add(hist_FG_like_local)
                    hist_BG_like.Add(hist_BG_like_local,scale)
                    hist_stat.Add(infile.Get(dirname+"PoolStatistics"))

        hists_ep_FG_read_centr.append(hist_FG)
        hists_ep_BG_read_centr.append(hist_BG)
        hists_ee_FG_read_centr.append(hist_FG_like)
        hists_ee_BG_read_centr.append(hist_BG_like)
        hist_stat_centr.append(hist_stat)
    hists_ep_FG_read.append(hists_ep_FG_read_centr)
    hists_ep_BG_read.append(hists_ep_BG_read_centr)
    hists_ee_FG_read.append(hists_ee_FG_read_centr)
    hists_ee_BG_read.append(hists_ee_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)

infile.Close()

['inv_mass_ee_DCA_V0_', 'inv_mass_ee_sDCA_V0_']
['inv_mass_ee_DCA_V1_', 'inv_mass_ee_sDCA_V1_']
['inv_mass_ee_DCA_V2_', 'inv_mass_ee_sDCA_V2_']
['inv_mass_ee_DCA_V3_', 'inv_mass_ee_sDCA_V3_']
['inv_mass_ee_DCA_V4_', 'inv_mass_ee_sDCA_V4_']
['inv_mass_ee_DCA_V5_', 'inv_mass_ee_sDCA_V5_']


In [5]:
iOption = 0
rebin = 1
new_m_array = array( 'd', [0.,0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.725,0.8,0.875,0.95,1.0,1.05,1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,3.55,3.65,3.75,3.85,4.5])
N_new_bins = len(new_m_array)-1
print(N_new_bins)

N_inv_m = 35  
inv_m_bins = array( 'd', [0.,0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.725,0.8,0.875,0.95,1.0,1.05,1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,3.55,3.65,3.75,3.85,4.5])

N_DCA = 42  
DCA_bins = array( 'd', [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,220,240,260,280,300,350,400,450,500,600,700,800,900,1000,1100,1200,1300,1400,1600,1800,2000,3000])
if is_new: DCA_bins = array( 'd', [0,2,5,10,15,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,180,200,220,240,260,300,350,400,450,500,600,700,800,900,1000,1100,1200,1300,1400,1600,1800,2000,3000])


N_sDCA = 35  
sDCA_bins = array( 'd', [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.2,2.4,2.6,2.8,3.0,3.5,4.0,4.5,5.0,6.0,7.0,8.0,9.0,10,20])

N_pt = 16
pt_array = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.2,1.4,1.6,2.0,2.5,3.0]


35


In [6]:
iOption = 4
rebin = 4
do_substract = 0
do_jpsi_m = 0
pt_low, pt_high = 1, 16
phi_v_low, phi_v_high = 16, 29
if do_jpsi_m: phi_v_low, phi_v_high = 23,29
print(inv_m_bins[phi_v_low-1],inv_m_bins[phi_v_high])
c3 = root.TCanvas(f"c2",f"c2",1500,1200)
c3.Divide(2,3)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG.Scale(1)

    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)

    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        proj_BG.SetBinError(ibin,(proj_BG.GetBinContent(ibin))**0.5)
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinContent(ibin,proj_BG.GetBinContent(ibin)/proj_BG.GetBinWidth(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinError(ibin,proj_BG.GetBinError(ibin)/proj_BG.GetBinWidth(ibin))

    Format_Hist_total(proj,"DCA, #mum","dN/DCA, #mum^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.2)

    Format_Graph(proj_BG,20,1.5,2,3,2)
    proj_BG.Scale(0.97)
    if do_substract: proj.Add(proj_BG,-1)
    print(round(proj.GetMean(),1))
    root.gPad.SetLogy(1)
    proj.Draw("p")
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    if not do_jpsi_m: proj.SetAxisRange(0,980)
    else : proj.SetAxisRange(0,580)
    if not do_substract: proj_BG.Draw("same p")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
    legends.append(root.TLegend(0.56,0.5,0.94,0.85,f"{inv_m_bins[phi_v_low-1]}"+"<m_{inv}<"+f"{inv_m_bins[phi_v_high]}"))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
if not do_jpsi_m: c3.Print("output/DCA/DCA_BG.png")
else : c3.Print("output/DCA/jpsi_DCA_BG.png")

1.15 3.3
83.7
84.0
82.7
77.5
72.6
60.0


In [24]:
iOption = 4
rebin = 4
do_substract = 1
do_jpsi_m = 0
pt_low, pt_high = 1, 16
phi_v_low, phi_v_high = 16, 29
if do_jpsi_m: phi_v_low, phi_v_high = 23,29
pt_bins=[1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,12,12,13,13,14,14,15,15,16,16]
print(inv_m_bins[phi_v_low-1],inv_m_bins[phi_v_high])
c3 = root.TCanvas(f"c2",f"c2",1500,1200)
c3.Divide(2,3)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)

    proj = hists_ep_FG_read[iOption][icentr].ProjectionY(f"py{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"py_BG_{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG.Add(proj_BG,-1)

    for ipt in range(int(len(pt_bins)/2)):
        proj2 = hists_ee_FG_read[iOption][icentr].ProjectionY(f"pxl{icentr}{iOption}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        proj2_BG = hists_ee_BG_read[iOption][icentr].ProjectionY(f"pxl_BG_{icentr}{iOption}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        left = proj2.FindBin(1.4)
        right = proj2.FindBin(1.8)+1

        scale = proj2.Integral(left,right)/(proj2_BG.Integral(left,right)+1)
        #if abs(scale - 1) > 0.1: print(icentr, ipt, scale)
        proj_BG.Add(hists_ep_BG_read[iOption][icentr].ProjectionY(f"pxll_BG_{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1]),scale)
    
    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        proj_BG.SetBinError(ibin,(proj_BG.GetBinContent(ibin))**0.5)
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinContent(ibin,proj_BG.GetBinContent(ibin)/proj_BG.GetBinWidth(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinError(ibin,proj_BG.GetBinError(ibin)/proj_BG.GetBinWidth(ibin))
    

    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)

    Format_Hist_total(proj,"DCA, #mum","dN/DCA, #mum^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.2)

    Format_Graph(proj_BG,20,1.5,2,3,2)
    proj_BG.Scale(0.999)
    if do_substract: proj.Add(proj_BG,-1)
    print(round(proj.GetMean(),1))
    root.gPad.SetLogy(1)
    proj.Draw("p")
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    if not do_jpsi_m: proj.SetAxisRange(0,980)
    else : proj.SetAxisRange(0,580)
    if not do_substract: proj_BG.Draw("same p")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
    legends.append(root.TLegend(0.56,0.5,0.94,0.85,f"{inv_m_bins[phi_v_low-1]}"+"<m_{inv}<"+f"{inv_m_bins[phi_v_high]}"))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
if not do_jpsi_m: c3.Print("output/DCA/DCA_BG.png")
else : c3.Print("output/DCA/jpsi_DCA_BG.png")

1.15 3.3
98.7
98.9
95.5
62.6
43.3
39.7


In [8]:
N_inv_m = 35
inv_m_bins = array( 'd', [0.,0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.725,0.8,0.875,0.95,1.0,1.05,1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,3.55,3.65,3.75,3.85,4.5])


In [9]:
c5 = root.TCanvas("ccc33","ccc33",1500,1200)
iOption = 0
rebin = 1
is_sub = 0
pt_low, pt_high = 1, 16
print(pt_array[pt_low],pt_array[pt_high])
c5.Divide(3,2)
legends =[]
projs,projs_BG=[],[]

used_m_bins = [19,19,20,21,22,24,25,28,29,30,31,34]
graphs= []
print(inv_m_bins[used_m_bins[0]-1],inv_m_bins[used_m_bins[1]],inv_m_bins[used_m_bins[2]-1],inv_m_bins[used_m_bins[3]],inv_m_bins[used_m_bins[4]-1],\
      inv_m_bins[used_m_bins[5]],inv_m_bins[used_m_bins[6]-1],inv_m_bins[used_m_bins[7]],inv_m_bins[used_m_bins[8]-1],inv_m_bins[used_m_bins[9]],\
        inv_m_bins[used_m_bins[10]-1],inv_m_bins[used_m_bins[11]])
for icentr in range(N_centr):
    c5.cd(icentr+1)
    proj0 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px00_0_{icentr}{iOption}",used_m_bins[2*0],used_m_bins[2*0+1],pt_low,pt_high) - is_sub * hists_ep_BG_read[iOption][icentr].ProjectionY(f"BG_px00_0_{icentr}{iOption}",used_m_bins[2*0],used_m_bins[2*0+1],pt_low,pt_high)
    proj1 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px11_0_{icentr}{iOption}",used_m_bins[2*1],used_m_bins[2*1+1],pt_low,pt_high) - is_sub * hists_ep_BG_read[iOption][icentr].ProjectionY(f"BG_px11_0_{icentr}{iOption}",used_m_bins[2*1],used_m_bins[2*1+1],pt_low,pt_high)
    proj2 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px22_0_{icentr}{iOption}",used_m_bins[2*2],used_m_bins[2*2+1],pt_low,pt_high) - is_sub * hists_ep_BG_read[iOption][icentr].ProjectionY(f"BG_px22_0_{icentr}{iOption}",used_m_bins[2*2],used_m_bins[2*2+1],pt_low,pt_high)
    proj3 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px33_0_{icentr}{iOption}",used_m_bins[2*3],used_m_bins[2*3+1],pt_low,pt_high) - is_sub * hists_ep_BG_read[iOption][icentr].ProjectionY(f"BG_px33_0_{icentr}{iOption}",used_m_bins[2*3],used_m_bins[2*3+1],pt_low,pt_high)
    proj4 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px44_0_{icentr}{iOption}",used_m_bins[2*4],used_m_bins[2*4+1],pt_low,pt_high) - is_sub * hists_ep_BG_read[iOption][icentr].ProjectionY(f"BG_px44_0_{icentr}{iOption}",used_m_bins[2*4],used_m_bins[2*4+1],pt_low,pt_high)
    proj5 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px55_0_{icentr}{iOption}",used_m_bins[2*5],used_m_bins[2*5+1],pt_low,pt_high) - is_sub * hists_ep_BG_read[iOption][icentr].ProjectionY(f"BG_px55_0_{icentr}{iOption}",used_m_bins[2*5],used_m_bins[2*5+1],pt_low,pt_high)
    print(round(proj0.GetMean(),1),round(proj0.GetMeanError(),1),round(proj1.GetMean(),1),round(proj1.GetMeanError(),1),round(proj2.GetMean(),1),round(proj2.GetMeanError(),1))
    print(round(proj3.GetMean(),1),round(proj3.GetMeanError(),1),round(proj4.GetMean(),1),round(proj4.GetMeanError(),1),round(proj5.GetMean(),1),round(proj5.GetMeanError(),1))
   

    projects = [proj0,proj1,proj2,proj3,proj4,proj5]
    #for proj in projects:
    #    for ibin in range (1,1+proj.GetNbinsX()):
    #        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
    graphs.append(root.TGraphErrors())
    for ipoint in range(0,6):
        graphs[-1].AddPoint(0.5*(inv_m_bins[used_m_bins[2*ipoint]-1]+inv_m_bins[used_m_bins[2*ipoint+1]]), projects[ipoint].GetMean())
        graphs[-1].SetPointError(ipoint,0.5*(inv_m_bins[used_m_bins[2*ipoint+1]]-inv_m_bins[used_m_bins[2*ipoint]-1]),projects[ipoint].GetMeanError())
    #proj_BG.Scale(1)
    Format_Hist_total(graphs[-1],"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.2)
    #Format_Graph(proj_BG,49,1.5,2,3,2)
    #root.gPad.SetLogy(1)
    graphs[-1].Draw("ap")
    #proj_BG.Draw("same p")
    #proj.Draw("same p")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    legends[-1].AddEntry(proj,"FG","pl")
    #legends[-1].AddEntry(proj_BG,"BG","pl")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c5.Draw()
c5.Print("output/m_ee/m_ee_newpt_FG_BG.png")

0.1 3.0
1.75 2.0 2.0 2.5 2.5 3.0 3.0 3.2 3.2 3.45 3.45 3.85
125.8 0.7 115.6 0.8 106.6 1.5
104.7 3.3 109.6 5.3 96.9 5.0
126.9 0.8 116.3 0.9 107.4 1.7
107.3 3.9 108.8 6.1 98.3 5.6
120.6 1.9 114.2 2.0 103.5 3.3
106.7 7.1 114.6 11.4 81.0 10.7
110.8 4.4 98.4 4.8 103.7 8.0
64.5 10.7 108.8 36.8 144.0 31.3
89.0 9.8 98.1 13.2 102.6 16.8
64.3 21.5 45.0 0.0 0.0 0.0
66.6 11.9 97.5 24.6 108.3 24.2
22.5 7.2 45.0 0.0 0.0 0.0


In [10]:
c5 = root.TCanvas("ccc33","ccc33",1500,1100)
iOption = 4
rebin = 2
is_sub = 0
pt_low, pt_high = 1, 16
print(pt_array[pt_low],pt_array[pt_high])
c5.Divide(3,2)
legends =[]
projs,projs_BG=[],[]

for icentr in range(N_centr):
    c5.cd(icentr+1)
    Nnn = N_DCA
    DCA_hist_2D_FG = root.TH2D(f"DCA_FG_{icentr}_{iOption}",f"DCA_FG_{icentr}_{iOption}",N_inv_m,inv_m_bins,N_DCA,DCA_bins)
    DCA_hist_2D_BG = root.TH2D(f"DCA_BG_{icentr}_{iOption}",f"DCA_BG_{icentr}_{iOption}",N_inv_m,inv_m_bins,N_DCA,DCA_bins)
    
    hists_out = [DCA_hist_2D_FG, DCA_hist_2D_BG]

    hists_in = [hists_ep_FG_read[iOption][icentr], hists_ep_BG_read[iOption][icentr]]
    
    sub = 0
    for iy in range(1, 1+Nnn):
        for ix in range(1,1+N_inv_m):
            for ihist in range(2):
                content = 0.    
                for iz in range(pt_low,1+pt_high):
                    content += hists_in[ihist].GetBinContent(ix,iy,iz)
                hists_out[ihist].SetBinContent(ix-sub,iy,hists_out[ihist].GetBinContent(ix-sub,iy)*0+ content)
    hists_out[0].Add(hists_out[1],-is_sub)
    proj = hists_out[0].ProfileX()
    proj_BG = hists_out[1].ProfileX()
    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)
    #proj.SetAxisRange(1.15,4.)
    proj_BG.Scale(1)
    Format_Hist_total(proj,"m_{ee}, GeV","#LT DCA #GT",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.2)
    Format_Graph(proj_BG,49,1.5,2,3,2)
    #root.gPad.SetLogy(1)
    #proj.Divide(proj_BG)
    proj.Draw("p")
    if not is_sub:proj_BG.Draw("same p")
    proj.Draw("same p")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    legends[-1].AddEntry(proj,"FG","pl")
    #legends[-1].AddEntry(proj_BG,"BG","pl")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c5.Draw()
c5.Print("output/DCA/DCA_FG_BG.png")

0.1 3.0


In [11]:
c5 = root.TCanvas("ccc33","ccc33",900,700)
iOption = 0
rebin = 1
is_sub = 0
pt_low, pt_high = 1, 16
print(pt_array[pt_low],pt_array[pt_high])
legends =[]
projs,projs_BG=[],[]

for icentr in range(1):
    c5.cd(icentr+1)
    Nnn = 42+0*N_DCA
    DCA_hist_2D_FG = root.TH2D(f"DCA_FG_{icentr}_{iOption}",f"DCA_FG_{icentr}_{iOption}",N_inv_m,inv_m_bins,N_DCA,DCA_bins)
    DCA_hist_2D_BG = root.TH2D(f"DCA_BG_{icentr}_{iOption}",f"DCA_BG_{icentr}_{iOption}",N_inv_m,inv_m_bins,N_DCA,DCA_bins)
    
    hists_out = [DCA_hist_2D_FG, DCA_hist_2D_BG]

    hists_in = [hists_ep_FG_read[iOption][icentr], hists_ep_BG_read[iOption][icentr]]
    
    sub = 0
    for iy in range(0, 1+Nnn):
        for ix in range(1,1+N_inv_m):
            for ihist in range(2):
                content = 0.    
                for iz in range(pt_low,1+pt_high):
                    content += hists_in[ihist].GetBinContent(ix,iy,iz)
                hists_out[ihist].SetBinContent(ix-sub,iy,hists_out[ihist].GetBinContent(ix-sub,iy)*0+ content)
    hists_out[0].Add(hists_out[1],-is_sub)
    proj = hists_out[0].ProfileX()
    proj_BG = hists_out[1].ProfileX()
    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)
    #proj.SetAxisRange(1.15,4.)
    #proj.Scale(0.02)
    #proj_BG.Scale(0.02)
    Format_Hist_total(proj,"m_{ee}, GeV","#LT DCA #GT",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.2)
    #proj.SetMaximum(1.65)
    Format_Graph(proj_BG,49,2.0,2,3,2)
    #root.gPad.SetLogy(1)
    #proj.Divide(proj_BG)
    proj.Draw("p")
    if not is_sub:proj_BG.Draw("same p")
    proj.Draw("same p")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.2,0.15,0.5,0.5,centralities[icentr]))
    legends[-1].AddEntry(proj,"FG","pl")
    #legends[-1].AddEntry(proj_BG,"BG","pl")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c5.Draw()
c5.Print("output/DCA/DCA.png")

0.1 3.0


In [12]:
#!jupyter nbconvert --to html m_ee_plotter.ipynb --HTMLExporter.theme=dark;